# Run analysis on multiple files

In [5]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import interpolate
import matplotlib.pyplot as plt 
import os
import pyfilaments.analysisutils as analysis
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing
import h5py

In [17]:
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/analysis_test'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/b0_2_1_FullSweep_Final'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/FinerResolution_BendingStiffness'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-06-23'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-08-18'
# data_folder = '/home/deepak/Dropbox/LacryModeling/ModellingResults/StochasticActivity/NormalDistribution'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/b0_2_1_FullSweep_Final/activity_time_750'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/FilamentLength_Sweeps/activity_time_750'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-08-28'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-10-19'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrength_sweep'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-11-08'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ConstantCompressiveActivity_sweep'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-11-25'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-12-02'

# Axial stiffness sweep
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrength_sweep_k_10_kappa_6.25'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrength_sweep_k_100_kappa_6.25'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrength_sweep_RandomAnglesIC'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/Analysis_test'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrengthSweeps_diffActivityTimescales/activity_timescale_700'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityTimeSweep_constantActivity_1.5'

data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/StochasticActivity/NormalDistribution'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/StochasticActivity/DeterministicActivity'

# Find all simulation data files and create a list
files_list = []
 # Walk through the folders and identify the simulation data files
for dirs, subdirs, files in os.walk(data_folder, topdown=False):

    root, subFolderName = os.path.split(dirs)

    for fileNames in files:
        if(fileNames.endswith('hdf5') and fileNames[0] != '.' and 'analysis' not in fileNames and 'ic' not in fileNames and 'eigenvectors' not in fileNames):
            files_list.append(os.path.join(dirs,fileNames))
#         if(fileNames.endswith('hdf5') and fileNames[0] != '.' and fileNames == 'SimResults_00.hdf5'):
#             files_list.append(os.path.join(dirs,fileNames))

print('No:of Simulation files: ', len(files_list))


    
N_ACTIVITY_CYCLES = 490 # No:of activity cycles requested

No:of Simulation files:  60


In [20]:
def run_filament_analysis(file):
    print('Analyzing file ...')
    print(file)

    filament = analysis.analysisTools(file = file)
    

    root_folder, file_name = os.path.split(file)
    
    # @@@ Better to use Metadata from the HDF5 file so there is no possible delinking of data and metadata
    df = pd.DataFrame({'N particles':[filament.Np],'radius':[filament.radius],'bond length':[filament.b0],
                       'spring constant':[filament.k], 'kappa_hat':[filament.kappa_hat], 
                       'force strength':[filament.F0], 'stresslet strength':[filament.S0], 
                       'potDipole strength':[filament.D0], 'simulation type':[filament.sim_type], 
                       'boundary condition 0':[filament.bc[0]], 'boundary condition 1':[filament.bc[-1]], 
                       ' activity time scale':[filament.activity_timescale], 'viscosity':[filament.mu]})
    
    periodic_flag = False 
    min_period = None
    threshold_index = int(0)  # Activity cycle when the dynamics transitions to a periodic cycle
    
    
    # Filament tip coverage analysis
    filament.filament_tip_coverage(save = True, overwrite = True)
    
    search_efficiency = filament.search_efficiency()
    
    
    # Check if simulation was completed or terminated before completion
    if(int(filament.Time[-1]) >= int(filament.activity_timescale*N_ACTIVITY_CYCLES)):
        simulation_completed = True
        # Classify the dynamics
        periodic_flag, min_period, threshold_index = filament.classify_filament_dynamics()
    else:
        simulation_completed = False
        
    
    activity_cycles_completed = int(filament.Time[-1]/filament.activity_timescale)
    
    if(periodic_flag == False):
        # if dynamics is aperiodic
        df['period'] = float(np.nan)
        df['threshold index'] = int(0)
    else:
        df['period'] = min_period
        df['threshold index'] = int(threshold_index)
        
        
    df['simulation completed'] = simulation_completed
    df['periodic dynamics'] = periodic_flag
    
    print('Periodic dynamics:', df['periodic dynamics'])
    print('Period:', df['period'])
    
    df['max unique locations'] = filament.derived_data['unique position count'][-1]
    df['activity cycles completed'] = activity_cycles_completed
    
    print('Activity cycles completed: {}'.format(activity_cycles_completed))
    
    df['search efficiency'] = search_efficiency
    
#     df['activity type'] =  filament.activity['type']
    
    # Calculate the filament tip angles (at constant phase) and save them to a file
    
#     phase = 0
#     angles = filament.compute_basetip_angle_at_constant_phase(phase_value = phase)
#     print(angles)
    
    
    # Save the analysis data
    save_file = file_name[:-5] + '_analysis.csv'
    save_folder = os.path.join(root_folder, 'Analysis')

    if(not os.path.exists(save_folder)):
        os.makedirs(save_folder)
        
    df.to_csv(os.path.join(save_folder, save_file))
    
    # Create a dataset and save the data
    save_file = file_name[:-5] + '_analysis.hdf5'


    with h5py.File(os.path.join(save_folder, save_file), "w") as f:
        dset = f.create_group("analysis data")
        dset.attrs['N particles'] = filament.Np
        dset.attrs['radius'] = filament.radius
        dset.attrs['bond length'] = filament.b0
        dset.attrs['spring constant'] = filament.k
        dset.attrs['kappa_hat'] = filament.kappa_hat
        dset.attrs['force strength'] = filament.F0
        dset.attrs['stresslet strength'] = filament.S0
        dset.attrs['potDipole strength'] = filament.D0
        dset.attrs['simulation type'] = filament.sim_type
        dset.attrs['activity time scale'] = filament.activity_timescale
        dset.attrs['viscosity'] = filament.mu
        dset.attrs['boundary condition 0'] = filament.bc[0]
        dset.attrs['boundary condition 1'] = filament.bc[-1]
        dset.attrs['threshold cycle'] = df['threshold index']  # This is the cause of the issue...
        
        print(periodic_flag)
        print(min_period)
        dset.attrs['activity cycles completed'] = activity_cycles_completed

        dset.attrs['Sim complete'] = int(simulation_completed)

        dset.attrs['periodic'] = df['periodic dynamics']
        dset.attrs['period'] =  df['period']
        
#         dset.attrs['constant phase'] = phase
#         dset.create_dataset("Tip angles", data = angles)

In [21]:
num_cores = multiprocessing.cpu_count()

num_cores = 12

results = Parallel(n_jobs=num_cores,  verbose=10)(delayed(run_filament_analysis)(file) for file in tqdm(files_list))


 80%|████████  | 48/60 [00:47<00:15,  1.29s/it][Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   47.3s
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:   59.8s
100%|██████████| 60/60 [00:59<00:00,  1.00it/s]
[Parallel(n_jobs=12)]: Done  44 out of  60 | elapsed:  1.1min remaining:   23.8s
[Parallel(n_jobs=12)]: Done  51 out of  60 | elapsed:  1.1min remaining:   12.1s
[Parallel(n_jobs=12)]: Done  58 out of  60 | elapsed:  1.2min remaining:    2.5s
[Parallel(n_jobs=12)]: Done  60 out of  60 | elapsed:  1.3min finished


In [21]:
print(np.linspace(0.5,3,40))

[0.5        0.56410256 0.62820513 0.69230769 0.75641026 0.82051282
 0.88461538 0.94871795 1.01282051 1.07692308 1.14102564 1.20512821
 1.26923077 1.33333333 1.3974359  1.46153846 1.52564103 1.58974359
 1.65384615 1.71794872 1.78205128 1.84615385 1.91025641 1.97435897
 2.03846154 2.1025641  2.16666667 2.23076923 2.29487179 2.35897436
 2.42307692 2.48717949 2.55128205 2.61538462 2.67948718 2.74358974
 2.80769231 2.87179487 2.93589744 3.        ]
